In [ ]:
# standard python utilities
import os
from os.path import join, basename,dirname, exists, expanduser
import sys
import glob
import pandas as pd
import numpy as np
import time

# standard python plotting utilities
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# standard geospatial python utilities
# import pyproj # for converting proj4string
# import shapely
import geopandas as gpd
# import rasterio

# mapping utilities
import contextily as ctx
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from matplotlib.ticker import MaxNLocator


In [ ]:
usr_dir = expanduser('~')
doc_dir = join(usr_dir, 'Documents')
    
# dir of all gwfm data
gwfm_dir = dirname(doc_dir)+'/Box/research_cosumnes/GWFlowModel'
# dir of stream level data for seepage study
proj_dir = gwfm_dir + '/Oneto_Denier/'
dat_dir = proj_dir+'Stream_level_data/'

fig_dir = proj_dir+'/Streambed_seepage/figures/'
hob_dir = join(gwfm_dir, 'HOB_data')
sfr_dir = gwfm_dir+'/SFR_data/'




In [ ]:
def add_path(fxn_dir):
    """ Insert fxn directory into first position on path so local functions supercede the global"""
    if fxn_dir not in sys.path:
        sys.path.insert(0, fxn_dir)

add_path(doc_dir+'/GitHub/flopy')
import flopy 
py_dir = join(doc_dir,'GitHub/CosumnesRiverRecharge/python_utilities')
add_path(py_dir)
from mf_utility import get_dates, get_layer_from_elev, clean_wb
from map_cln import gdf_bnds, plt_cln
from hyd_utility import dt_2_wy

# from importlib import reload
# import mf_utility
# reload(mf_utility)

In [ ]:
# scenario specific function
from OD_utility import run_stats

In [ ]:
# scenario = '' # baseline, levee removal occurred in 2014
# create identifier for scenario if levee removal didn't occur
scenario = 'no_reconnection'

In [ ]:
ext_dir = 'F:/WRDAPP'
c_dir = 'C:/WRDAPP'
if os.path.exists(ext_dir):
    loadpth = ext_dir 
elif os.path.exists(c_dir):
    loadpth = c_dir 
loadpth +=  '/GWFlowModel/Cosumnes/Stream_seepage'

upscale = 'upscale4x_'
# model_nam = 'oneto_denier_'+upscale+'2014_2018'
model_nam = 'oneto_denier_'+upscale+'2014_2020'
model_ws = join(loadpth,model_nam)

if scenario != '':
    model_ws += '_' + scenario
    
# model_ws = join(loadpth,'parallel_oneto_denier','realization000')
load_only = ['DIS','UPW','SFR','OC', "EVT",'LAK']
m = flopy.modflow.Modflow.load('MF.nam', model_ws= model_ws, 
                                exe_name='mf-owhm.exe', version='mfnwt',
                              load_only=load_only,
                              )


In [ ]:
label_baseline = 'Baseline'
label_restoration = 'Levee removal'

In [ ]:
nrow, ncol = (m.dis.nrow, m.dis.ncol)

In [ ]:
model_ws0 = join(loadpth,model_nam)

In [ ]:
print('Quantiles: ',[0,0.5,0.6,0.75,1])
print('HK :',np.quantile(m.upw.hk.array,[0,0.5,0.6,0.75,1]))
print('VKA :',np.quantile(m.upw.vka.array,[0,0.5,0.6,0.75,1]))

In [ ]:
model_grp = 'inset_oneto_denier'
grid_dir = join(gwfm_dir, 'DIS_data/streambed_seepage/grid')
grid_fn = join(grid_dir, model_grp,'rm_only_grid.shp')
grid_p = gpd.read_file(grid_fn)
grid_p.crs='epsg:32610'
m_domain = gpd.GeoDataFrame(pd.DataFrame([0]), geometry = [grid_p.unary_union], crs=grid_p.crs)

In [ ]:
XSg = pd.read_csv(join(model_ws,'04_XSg_filled.csv'))
XSg = gpd.GeoDataFrame(XSg, geometry = gpd.points_from_xy(XSg.Easting, XSg.Northing), crs='epsg:32610')

# overwrite SFR segment/reach input relevant to seepage
# sensor_dict = pd.read_csv(join(model_ws, 'sensor_xs_dict.csv'), index_col=0)
# XS_params = sensor_dict.join(params.set_index('Sensor'), on='Sensor')

In [ ]:
params = pd.read_csv(model_ws+'/ZonePropertiesInitial.csv', index_col='Zone')
# convert from m/s to m/d
params['K_m_d'] = params.K_m_s * 86400 
vka = m.upw.vka.array
tprogs_vals = np.arange(1,5)
tprogs_hist = np.flip([0.590, 0.155, 0.197, 0.058])
tprogs_quants = 1-np.append([0], np.cumsum(tprogs_hist)/np.sum(tprogs_hist))
vka_quants = pd.DataFrame(tprogs_quants[1:], columns=['quant'], index=tprogs_vals)
# dataframe summarizing dominant facies based on quantiles
vka_quants['vka_min'] = np.quantile(vka, tprogs_quants[1:])
vka_quants['vka_max'] = np.quantile(vka, tprogs_quants[:-1])
vka_quants['facies'] = params.loc[tprogs_vals].Lithology.values

In [ ]:
sfrdf = pd.DataFrame(m.sfr.reach_data)
grid_sfr = grid_p.set_index(['row','column']).loc[list(zip(sfrdf.i+1,sfrdf.j+1))].reset_index(drop=True)
grid_sfr = pd.concat((grid_sfr,sfrdf),axis=1)
# group sfrdf by vka quantiles
sfr_vka = vka[grid_sfr.k, grid_sfr.i, grid_sfr.j]
for p in vka_quants.index:
    facies = vka_quants.loc[p]
    grid_sfr.loc[(sfr_vka< facies.vka_max)&(sfr_vka>= facies.vka_min),'facies'] = facies.facies
#     # add color for facies plots
# grid_sfr = grid_sfr.join(gel_color.set_index('geology')[['color']], on='facies')

In [ ]:
lak_shp = join(gwfm_dir,'LAK_data/floodplain_delineation')
# shapefile rectangle of the area surrounding the Dam within about 5 cells
lak_gpd = gpd.read_file(join(lak_shp,'LCRFR_ModelDom_2017/LCRFR_2DArea_2015.shp' )).to_crs('epsg:32610')

lak_cells = gpd.sjoin(grid_p,lak_gpd,how='right',predicate='within').drop(columns='index_left')

# filter zone budget for Blodgett Dam to just within 5 cells or so of the Dam
zon_lak = np.zeros((grid_p.row.max(),grid_p.column.max()),dtype=int)
zon_lak[lak_cells.row-1,lak_cells.column-1]=1

zon_mod = np.ones((grid_p.row.max(),grid_p.column.max()),dtype=int)

In [ ]:
zon_color_dict = pd.read_excel('mf_wb_color_dict.xlsx',sheet_name='owhm_wb_dict', header=0, index_col='flux',comment='#').color.to_dict()
zon_name_dict = pd.read_excel('mf_wb_color_dict.xlsx',sheet_name='owhm_wb_dict', header=0, index_col='flux',comment='#').name.to_dict()

zb_alt = pd.read_excel('mf_wb_color_dict.xlsx',sheet_name='flopy_to_owhm', header=0, index_col='flopy',comment='#').owhm.to_dict()


## Sensor data and XS data

In [ ]:
rm_grid = pd.read_csv(join(proj_dir, 'mw_hob_cleaned.csv'))
rm_grid = gpd.GeoDataFrame(rm_grid, geometry = gpd.points_from_xy(rm_grid.Longitude,rm_grid.Latitude), 
                           crs='epsg:4326').to_crs(grid_p.crs)
# get model layer for heads
hob_row = rm_grid.row.values-1
hob_col = rm_grid.column.values-1

In [ ]:
gwl_long = pd.read_csv(join(model_ws,'gwl_long.csv'), parse_dates=['dt'])

In [ ]:
# XS are every 100 m
xs_all = pd.read_csv(dat_dir+'XS_point_elevations.csv',index_col=0)
xs_all = gpd.GeoDataFrame(xs_all,geometry = gpd.points_from_xy(xs_all.Easting,xs_all.Northing), crs='epsg:32610')


In [ ]:

# correspond XS to sensors
rm_elev = gpd.sjoin_nearest(XSg, rm_grid, how='right',lsuffix='xs', rsuffix='rm')
#MW_11, MW_CP1 had doubles with sjoin_nearest due to XS duplicates from Oneto_Denier
rm_elev = rm_elev.drop_duplicates(['xs_num','Sensor'])

## Model output - time variant

In [ ]:
hdobj0 = flopy.utils.HeadFile(model_ws0+'/MF.hds')
hdobj = flopy.utils.HeadFile(model_ws+'/MF.hds')
spd_stp = hdobj.get_kstpkper()
times = hdobj.get_times()


In [ ]:
strt_date, end_date, dt_ref = get_dates(m.dis, ref='strt')


# HOB

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

def nse(targets,predictions):
    return 1-(np.sum((targets-predictions)**2)/np.sum((targets-np.mean(predictions))**2))

def clean_hob(model_ws, dt_ref):
    hobout = pd.read_csv(join(model_ws,'MF.hob.out'),delimiter=r'\s+', header = 0,names = ['sim_val','obs_val','obs_nam'],
                         dtype = {'sim_val':float,'obs_val':float,'obs_nam':object})
    hobout[['Sensor', 'spd']] = hobout.obs_nam.str.split('p',n=2, expand=True)
    hobout['kstpkper'] = list(zip(np.full(len(hobout),0), hobout.spd.astype(int)))
    hobout = hobout.join(dt_ref.set_index('kstpkper'), on='kstpkper')
    hobout.loc[hobout.sim_val.isin([-1e30, -999.99,-9999]), 'sim_val'] = np.nan
    hobout = hobout.dropna(subset='sim_val')
    hobout['error'] = hobout.obs_val - hobout.sim_val
    hobout['sq_error'] = hobout.error**2
    
    return(hobout)


In the final iteration of the model, Oneto-Ag isn't fit that badly anymore likely because of the aquifer thickening and updated Kx, adjusting Ss would improve further perhaps, but since the focus is the shallow network we will leave it out.

In [ ]:
hobout = clean_hob(model_ws, dt_ref)
# removing oneto ag because of large depth offset
hobout = hobout[hobout.Sensor != 'MW_OA']

In [ ]:
hobout0 = clean_hob(model_ws0, dt_ref)
# removing oneto ag because of large depth offset
hobout0 = hobout0[hobout0.Sensor != 'MW_OA']

In [ ]:
def return_stat(hobout):
    r2 = r2_score(hobout.obs_val, hobout.sim_val)
    RMSE = mean_squared_error(hobout.obs_val, hobout.sim_val, squared=False) # false returns RMSE instead of MSE
    NSE = nse(hobout.obs_val, hobout.sim_val)
    return(r2, RMSE, NSE)

In [ ]:
return_stat(hobout0), return_stat(hobout)

In [ ]:
hob_long = hobout.join(hobout0.set_index('obs_nam')[['sim_val']], on='obs_nam',rsuffix='0')
hob_long = hob_long.melt(id_vars=['dt', 'Sensor'],value_vars=['sim_val','obs_val','sim_val0'],
                         value_name='gwe', var_name='type')

# hob_long = hobout.melt(id_vars=['dt', 'Sensor'],value_vars=['sim_val','obs_val'], value_name='gwe', var_name='type')


In [ ]:
# g = sns.relplot(hob_long[hob_long.Sensor.isin(['MW_2','MW_3'])], x='dt',y='gwe', 
#                 row='Sensor',hue = 'type', kind='line')


In [ ]:
# hob_long, x='dt',y='
g = sns.relplot(hob_long, x='dt',y='gwe',col='Sensor',hue = 'type',  col_wrap=4, kind='line')

axes = g.axes.flatten()
mw = hob_long.Sensor.unique()

for n in np.arange(0,len(axes)):
    mw_dat = rm_elev[rm_elev.Sensor ==mw[n]]
    axes[n].axhline(mw_dat['MPE (meters)'].values[0], ls='--', linewidth=3, color='brown')
    axes[n].axhline(mw_dat['z_m_min_cln'].values[0]-1, ls='--', linewidth=3, color='blue')
    # axes[n].axhline(mw_dat['bot_screen_m'].values[0]-1, ls='--', linewidth=3, color='black')

In [ ]:
gage_cols = ['time','stage','volume','conc','inflows','outflows','conductance','error']

def read_gage(gagenam):
    gage = pd.read_csv(gagenam,skiprows=1, delimiter = r'\s+', engine='python')
    cols = gage.columns[1:-1]
    gage = gage.dropna(axis=1)
    gage.columns = cols
    strt_date = pd.to_datetime(m.dis.start_datetime)
    # the lake output includes the initial conditions (time==0)
    gage = gage[gage.Time >0]
    gage['dt'] = strt_date+((gage.Time-1)*24).astype('timedelta64[h]')
    gage = gage.set_index('dt')
    gage['dVolume'] = gage.Volume.diff()
    gage['Total_In'] = gage[['Precip.','Runoff','GW-Inflw','SW-Inflw']].sum(axis=1)
    gage['Total_Out'] = gage[['Evap.','Withdrawal','GW-Outflw','SW-Outflw']].sum(axis=1)
    gage['In-Out'] = gage.Total_In - gage.Total_Out
    gage['Error'] = gage['In-Out'] - gage['dVolume']
    # remove the steady state depth (dry start)
    gage.loc[gage['Stage(H)']<gage['Stage(H)'].quantile(0.01), 'Stage(H)'] = gage['Stage(H)'].quantile(0.01)
    # approximate depth, may not always work
    gage['depth'] = gage['Stage(H)']- gage['Stage(H)'].quantile(0.05)
    gage.loc[gage.depth<0,'depth']= 0
#     gage['name'] = run
    return(gage)


In [ ]:
lak_out0 = read_gage(join(model_ws0, 'MF_lak.go'))
lak_out = read_gage(join(model_ws, 'MF_lak.go'))


In [ ]:
# the days with WB error of 100% actually have no issues in the water budget
# lak_out[lak_out['Percent-Err']==100]

In [ ]:
# there are about 40 days with about 100% error or -60% error, the rest are well below 1%
# the error seems to happen when there is very little flow in the system rather than too much
# lak_out.plot(y='Percent-Err')
# cols = ['Percent-Err','GW-Outflw','SW-Inflw','SW-Outflw', 'In-Out']
# fig,ax = plt.subplots(len(cols),1,sharex=True)
# for n, col in enumerate(cols):
#     lak_out.plot(y=col, ax=ax[n])
# lak_out.columns
# plt.ylim(-1, 1)

Explain the mechanism of the results:
- a lower flow threshold and greater flow fraction puts more flow onto the floodplain so there is greater depth
- greater depth and area increases floodplain recharge
- the groundwater inflow is very small (1E3) compared to the outflow (1E5)
- would be good to plot groundwater elevation as well to highlight why the groundwater outflow changes

In [ ]:
lakarr = m.lak.lakarr.array

In [ ]:
# find where lake existed
lak_lay, lak_row, lak_col = np.where(lakarr[0]==1)
lak_kij = pd.DataFrame(np.transpose(np.where(m.lak.lakarr.array[0]==1)), columns=['k','i','j'])
# get first layer below lake cells
lak_kij = (lak_kij.groupby(['i','j']).max()+1).reset_index()
# create tuples for sampling
lak_idx = list(zip(lak_kij.k, lak_kij.i, lak_kij.j))

In [ ]:
def get_lak_head(hdobj, lak_idx):
    """
    Return the spatially averaged head for the maximum head at the input locations (idx)
    hdobj: flopy head object
    idx: list of tuples as (layer, row, column)
    """
    # get heads under the lake
    lak_ts = hdobj.get_ts(lak_idx)
    lak_ts_df = pd.DataFrame(lak_ts, columns=['totim']+lak_idx)
    lak_ts_df = lak_ts_df.set_index('totim')
    lak_ts_df = lak_ts_df.melt(ignore_index=False)
    lak_ts_df[['k','i','j']] = lak_ts_df.variable.tolist()
    lak_ts_df = lak_ts_df.drop(columns='variable') # drop to speed up groupby
    lak_head = lak_ts_df.groupby(['totim','i','j']).max().groupby('totim').mean()
    return lak_head

In [ ]:
lak_head = get_lak_head(hdobj, lak_idx)
lak_head0 = get_lak_head(hdobj0, lak_idx)

In [ ]:
import matplotlib.ticker as mtick


In [ ]:
cols = ['SW-Inflw', 'Stage(H)','GW-Outflw']
labels=['SW Inflow \n(million $m^3/d$)', 'Floodplain\nInundation\nStage (m)', 'GW Recharge \n(million $m^3/d$)']

scale = 1E6
fig,ax = plt.subplots(len(cols)+1,1, figsize=(6.5,6.5), dpi=300, sharex=True)
for n, col in enumerate(cols):
    if col in ['SW-Inflw','GW-Outflw']:
        scale = 1E6
    else:
        scale = 1
    lak_out0[col].multiply(1/scale).plot(y=col, ax=ax[n], legend=False, label=label_baseline)
    lak_out[col].multiply(1/scale).plot(y=col, ax=ax[n],legend=False, alpha=0.7, label='No Reconnection')
    ax[n].set_ylabel(labels[n])

# ax[0].yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1e'))
# ax[2].yaxis.set_major_formatter(mtick.FormatStrFormatter('%.1e'))

ax[-1].plot(lak_out.index, lak_head0['value'].values)
ax[-1].plot(lak_out.index, lak_head['value'].values,alpha=0.7)
ax[-1].set_ylabel('GW Elevation (m)')
# fig.legend([label_restoration,'Baselien'], ncol=2, loc='outside upper center', bbox_to_anchor=(0.5, 1.05),)
ax[0].legend([label_restoration,label_baseline], ncol=2, loc='upper right')
fig.tight_layout(h_pad=-0.1)
plt.xlabel('Datetime')

## Output reference values  
We need to supply numeric values to use in the result section to support comments on the differences between the baseline and restoration scenarios:  
- days with stream-floodplain connection in wet (2017, 2019) vs dry years
- the increase in days with connection from baseline to restoration and increase in depth (on average, percentage?)
- increase in total recharge on average
- increase in depth for number of days in wet year

In [ ]:
def fld_chk(lak_out0):
    lak_out0['wy'] = lak_out0.index.year
    lak_out0.loc[lak_out0.index.month>=10,'wy']+=1
    fld_chk = lak_out0.copy()[['GW-Outflw', 'SW-Inflw','depth','wy']]
    # fld_chk['fld'] = fld_chk.depth> 0 # only consider depths greater than 1 ft, Salmon passage?
    fld_chk['fld'] = fld_chk.depth> 1*0.3048 # only consider depths greater than 1 ft, Salmon passage?
    fld_chk['sw_in'] = fld_chk['SW-Inflw']>0 # any inflow
    fld_chk['rch_in'] = fld_chk['GW-Outflw']>0 # any inflow
    fld_chk[fld_chk==0] = np.nan
    fld_chk_out =fld_chk.groupby('wy').sum()[['fld','sw_in', 'rch_in']]
    mean_chk = fld_chk.groupby('wy').mean()[['GW-Outflw', 'SW-Inflw','depth']]
    return pd.concat((fld_chk_out, mean_chk),axis=1)

In [ ]:
# lak_out0

In [ ]:
# comparison of the number of days
fld_chk(lak_out0).mean()[['fld','depth']], fld_chk(lak_out).mean()[['fld','depth']]
# fld_chk(lak_out0).loc[[2015,2016,2018,2020]].mean()[['fld','sw_in']], fld_chk(lak_out0).loc[[2017, 2019]].mean()[['fld','sw_in']]

In [ ]:
fld_chk(lak_out).loc[[2015,2016,2018,2020]].mean(), fld_chk(lak_out).loc[[2017, 2019]].mean()

In [ ]:

gw_rch_inc = (fld_chk(lak_out0).mean()['GW-Outflw']- fld_chk(lak_out).mean()['GW-Outflw'])/fld_chk(lak_out).mean()['GW-Outflw']
print('Average increase in gw recharge is %.1f %%' %(100*gw_rch_inc))
rch_days_inc = fld_chk(lak_out0).mean()['rch_in']/fld_chk(lak_out).mean()['rch_in']
print('Average increase of %.2f times of days with recharge' %rch_days_inc)

# impact on depth and recharge
fld_chk(lak_out0).mean()['depth'], fld_chk(lak_out).mean()['depth']

In [ ]:
gwl_diff = (lak_head0 - lak_head)['value'].mean()
print('GW levels are %.2f m higher with restoration' %gwl_diff)

In [ ]:
# plot trend of groundwater elevations
gwl_diff_dt = (lak_head0 - lak_head)[['value']]
gwl_diff_dt['dt'] = dt_ref['dt'].values
gwl_diff_dt.set_index('dt').resample('AS-Oct').mean().plot()

# Identify location and timing of active ET 
Because it isn't easy to map ET, we can do an alternative mapping with head to show it is above the rooting depth.

In [ ]:
# load cbb file from any model scneario to get listing of kstpkper
cbc = join(m.model_ws, 'MF.cbc')

# zon_mod = np.ones((nrow,ncol),dtype=int)
# zb = flopy.utils.ZoneBudget(cbc, zon_mod)
# kstpkper = zb.kstpkper


In [ ]:
et_surf = m.evt.surf.array[0,0]
ext_dp = m.evt.exdp.array[0,0]
# bottom elevation of roots
et_botm =et_surf - ext_dp

et_row, et_col = np.where(ext_dp>2)

In [ ]:
# evtr_avg = m.evt.evtr.array.sum(axis=0)[0]

In [ ]:
et_lay = get_layer_from_elev(et_botm[et_row, et_col], m.dis.botm[:, et_row, et_col], m.dis.nlay)
# et_lay

In [ ]:
# identify which lake rows and columns are in GDEs
gde_lak_bool = (ext_dp>2)[lak_row, lak_col]
gde_lak_row = lak_row[gde_lak_bool]
gde_lak_col = lak_col[gde_lak_bool]

In [ ]:
head = hdobj.get_data(dt_ref.kstpkper.iloc[0])
head = np.ma.masked_where(head==-999.99, head)

In [ ]:
def find_active_ET(model_ws):
    hdobj = flopy.utils.HeadFile(join(model_ws, 'MF.hds'))
    et_act = np.zeros((m.dis.nper, nrow,ncol))
    
    for t in np.arange(0,m.dis.nper):
        head = hdobj.get_data(dt_ref.kstpkper.iloc[t])
        head = np.ma.masked_where(head==-999.99, head)
        # identify which GDE ET cells would active based on head
        b = head[et_lay, et_row, et_col] > et_botm[et_row, et_col]
        et_act[t, et_row[b], et_col[b]] = 1
    return et_act


In [ ]:
et_act = find_active_ET(model_ws)

In [ ]:
et_act0 = find_active_ET(model_ws0)

There is very little difference in the spatial location of ET between the scenarios, there are a few scattered dots around the edges of the GDE ET sites that would occur under the baseline scenario but not the no reconnection.  

What is more interesting is the number of days that are active is different. In general there tends to be 20% of the number of days more active ET in the restoration on the western side of oneto denier. The change is larger on the western side because the eastern side of the reconnected floodplain has the recharge due to the river so it is the western side that is dependent on floodplain inundation.  

The spots with greatest ET rates are the drainage areas (not because there is more simulated water there which in reality there is) but because the rooting depth of the GW ET is deeper than elsewhere.

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(9,3), sharey=True, layout='constrained')
im = ax[0].imshow(et_act.mean(axis=0))
plt.colorbar(im, shrink=0.5)# plt.show()

loc_diff = (et_act0.mean(axis=0)>0).astype(int)-(et_act.mean(axis=0)>0).astype(int)
im = ax[1].imshow(loc_diff)
plt.colorbar(im, shrink=0.5)
# plt.colorbar()
# plt.show()

im = ax[2].imshow((et_act0- et_act ).mean(axis=0))
plt.colorbar(im, shrink=0.5)

To assess the practial value of recharge on GDEs we should groupby the year and season to determine if there are more days in the season with ET to prevent die. The comparison should be cell by cell and then aggregated to overall. 
- the restoration keeps the ET going longer in wet years (2017, 2019) and creates it where it didn't exist in years like 2016, 2020 where there are only small events. The years like 2020 are what is important because it keeps there from being a total dead season, I'm not sure how much value there is in the wet years but maybe the extra vegetation growth is critical?

In [ ]:
def fp_et(et_act, lak_row, lak_col, dt_ref):
    et_df = pd.DataFrame(et_act[:, lak_row, lak_col], index=dt_ref.dt)
    et_df = et_df.melt(var_name='lak_id', value_name='et_bool', ignore_index=False)
    # et_yr = et_df.groupby('lak_id').resample('AS-Oct').sum().drop(columns=['lak_id'])
    et_yr = et_df.groupby('lak_id').resample('MS').sum().drop(columns=['lak_id'])
    
    et_yr = et_yr.reset_index(level='lak_id')
    return(et_yr)
lak_cols = ['lak_id','lak_row','lak_col']
lak_df = pd.DataFrame(np.transpose((np.arange(0,len(lak_row)), lak_row, lak_col)), 
                      columns=lak_cols)
# et_df = et_df.join( lak_df, on='lak_id')

In [ ]:
et_yr = fp_et(et_act, lak_row, lak_col, dt_ref)
et_yr0 = fp_et(et_act0, lak_row, lak_col, dt_ref)


In [ ]:
fig,ax = plt.subplots()
et_yr0[et_yr.lak_id==100].plot(y='et_bool',ax=ax, label=label_restoration)
et_yr[et_yr.lak_id==100].plot(y='et_bool',ax=ax,label=label_baseline)


In [ ]:
def make_et_df(et_act, et_act0, lak_row, lak_col, dt_ref):
    et_fp = pd.Series(et_act[:, lak_row, lak_col].mean(axis=1))
    et_fp0 = pd.Series(et_act0[:, lak_row, lak_col].mean(axis=1))
    et_df = pd.concat((et_fp0, et_fp), axis=1)
    et_df.index = dt_ref.dt
    et_df.columns=[label_restoration, label_baseline]
    return(et_df)
et_df = make_et_df(et_act, et_act0, gde_lak_row, gde_lak_col, dt_ref)
et_df['wy'] = dt_2_wy(et_df.index)
scen = [label_restoration, label_baseline]
et_df.plot(y=scen)

plt.ylabel('Floodplain GDE fraction with ET')
plt.xlabel('Date')

In [ ]:
# for WY2017 and Wy2018 using 0.5 as a threshold, compare how much longer it holds half of the maximum


# et_frac_max = et_df.max() # maximum fraction of floodplain covered
# et_frac_max
et_half = et_df[scen]>0.5
et_half['wy'] = et_df.wy
# identify the change in the number of days with half the floodplain GDEs active
et_change = et_half.groupby('wy').sum()
def print_diff(et_change, wy):
    diff_days = et_change.loc[wy, label_restoration]-et_change.loc[wy, label_baseline]
    print('There is a change of %.i days' %diff_days, 'in WY %.i' %wy)
print_diff(et_change, 2017)
print_diff(et_change, 2019)
print_diff(et_change, 2020)

The GSP evaluates GDEs with spring and fall groundwater contours and found difference of 0.4-7% in terms of GDE area. My results seem to show a much larger difference because I'm assuming different rooting depths (e.g., 2 - 10 m range) instead of a static 10 m depth.
- at least one lake cell had ET for up to 362 days in a year. t
- here are a few lake cells that have >300 days per year in the wetter years. Could use

In [ ]:
chk = et_yr0.groupby('lak_id').resample('AS-Oct').sum()
chk[chk.et_bool>300]

# Lak error checking

In [ ]:
# the flow in segment 32 is appropriately half of 31 and is equal to the lake inflow which shows the diversion is working
# sfrdf_full[sfrdf_full.segment==32].loc[plt_dates].plot(y=['Qin','Qout'])
# the lake out gets more storage built up which is then released to the stream
# lak_out.loc[plt_dates].iloc[70:110].plot(y=['SW-Inflw', 'SW-Outflw', 'dVolume'])
# lak_out0.loc[plt_dates].iloc[70:110].plot(y=['SW-Inflw', 'SW-Outflw', 'dVolume'])
# lak_out

## Understand impact on streamflow

In [ ]:
import mf_utility
from importlib import reload
reload(mf_utility)
from mf_utility import clean_sfr_df

In [ ]:
grid_sfr = pd.read_csv(join(model_ws,'grid_sfr.csv'),index_col=0)
# grid_sfr = grid_sfr[grid_sfr.strhc1!=0]
# grid_sfr['vka'] = vka[grid_sfr.k, grid_sfr.i, grid_sfr.j]

pd_sfr = grid_sfr.set_index(['iseg','ireach'])[['rchlen','strtop', 'facies', 'strthick']]
pd_sfr['Total distance (m)'] = pd_sfr['rchlen'].cumsum()
sfrdf =  clean_sfr_df(model_ws, dt_ref, pd_sfr, name='MF')
# gradient is stage - Ha/str thick, and strthick=1
sfrdf['h_aquifer'] = -(sfrdf.gradient*sfrdf.strthick - sfrdf.stage)

In [ ]:
grid_sfr = pd.read_csv(join(model_ws0,'grid_sfr.csv'),index_col=0)
# drop_iseg = grid_sfr[grid_sfr.strhc1==0].iseg.values
# grid_sfr = grid_sfr[grid_sfr.strhc1!=0]
grid_sfr['vka'] = vka[grid_sfr.k, grid_sfr.i, grid_sfr.j]
pd_sfr = grid_sfr.set_index(['iseg','ireach'])[['rchlen','strtop', 'facies', 'strthick']]
pd_sfr['Total distance (m)'] = pd_sfr['rchlen'].cumsum()

sfrdf0=  clean_sfr_df(model_ws0, dt_ref, pd_sfr, name='MF')
# gradient is stage - Ha/str thick, and strthick=1
sfrdf0['h_aquifer'] = -(sfrdf0.gradient*sfrdf0.strthick - sfrdf0.stage)


In [ ]:
wb, out_cols, in_cols = clean_wb(model_ws, dt_ref)
wb0, out_cols, in_cols = clean_wb(model_ws0, dt_ref)


In [ ]:
# plot cumulative seepage to see if it is the cause in outflow difference
def get_net_seep(wb):
    net_seep = wb.SFR_IN- wb.SFR_OUT + wb.LAK_IN - wb.LAK_OUT
    return(net_seep)
net_seep = get_net_seep(wb)
net_seep0 = get_net_seep(wb0)
# fig,ax = plt.subplots()
# net_seep0.plot(ax=ax, label=label_restoration)
# net_seep.plot(ax=ax, label=label_baseline)
# # net_seep0.resample('AS').sum().plot(ax=ax, label=label_restoration)
# # net_seep.resample('AS').sum().plot(ax=ax, label=label_baseline)
# ax.legend()

The model adds 29,458 m3 to the volume on 2014-10-22 before which the model reported 100% error so it may have been a MODFLOW desire to have some lake volume rather than dry, that volume persists in the lake as the dry volume so even when the lake fills and dries it returns to that volume. 

In [ ]:
# vol_chk = (lak_out0['dVolume'] - lak_out0['SW-Inflw'] +lak_out0['SW-Outflw'] +lak_out0['GW-Outflw']) - lak_out0['GW-Inflw']
# # vol_chk = (lak_out['dVolume'] - lak_out['SW-Inflw'] +lak_out['SW-Outflw'] +lak_out['GW-Outflw']) - lak_out['GW-Inflw']
# # vol_chk.plot()
# lak_out[vol_chk>0]
# lak_out['2014-10-20':'2015-4-30'].plot(y='Volume')

# Review this
It's still not clear what's wrong in the water budget for sfr to have more flow out when there is more seepage loss and more lake storage

In [ ]:
# can't use seven day roling average because this smooths out some key points
# plt_dates = pd.date_range('2020-1-1','2020-9-30')
plt_dates = pd.date_range('2016-1-1','2016-9-30')

fig, axes = plt.subplots(4,1, figsize=(6.5,6.5),dpi=300, sharex=True)
seg_plt = (sfrdf.segment==sfrdf.segment.max())
# seg_plt = (sfrdf.segment==sfrdf.segment.median())
# seg_plt = (sfrdf.segment==33)

ax = axes[0]
sfrdf0[seg_plt].loc[plt_dates].plot(y='Qout', ax=ax, label=label_restoration,linewidth=0.5)
sfrdf[seg_plt].loc[plt_dates].plot(y='Qout', ax=ax, label=label_baseline,linewidth=0.5)
sfrdf[sfrdf.segment==1].loc[plt_dates].plot(y='Qin', ax=ax, label='Model Inflow', linewidth=0.5)

ax = axes[1]
net_seep0.loc[plt_dates].plot(ax=ax, label=label_restoration,linewidth=0.5)
net_seep.loc[plt_dates].plot(ax=ax, label=label_baseline,linewidth=0.5)
ax.legend()

ax = axes[2]
lak_out0.loc[plt_dates].plot(y='dVolume', ax=ax, legend=False, label=label_restoration,linewidth=0.5)
lak_out.loc[plt_dates].plot(y='dVolume', ax=ax, legend=False, alpha=0.7, label=label_baseline,linewidth=0.5)
ax.legend()

ax = axes[3]
# model inflow minus loss to lake storage and groundwater (via stream and lake) should be outflow
chk = sfrdf[sfrdf.segment==1].Qin - lak_out.dVolume - net_seep
chk0 = sfrdf0[sfrdf0.segment==1].Qin - lak_out0.dVolume - net_seep0
chk = sfrdf[sfrdf.segment==1].Qin - lak_out['SW-Inflw'] +lak_out['SW-Outflw'] - net_seep
chk0 = sfrdf0[sfrdf0.segment==1].Qin - lak_out0.dVolume - net_seep0
chk0.loc[plt_dates].plot(ax=ax,linewidth=0.5, label=label_restoration)
chk.loc[plt_dates].plot(ax=ax,linewidth=0.5, label=label_baseline)
ax.legend()

# plt.yscale('log')
axes[0].set_ylabel('Outlet\nStreamflow\n($m^3/day$)')
axes[1].set_ylabel('Seepage ($m^3/day$)')
axes[2].set_ylabel('Change in \n Lake Volume \n($m^3/day$)')
axes[3].set_ylabel('WB Check \n(Qin - $\Delta$vol. Lake \n- net seep\n ($m^3/day$)')

plt.xlabel('Date')
# plt.xlim('2018-1-1','2018-3-1')

Final thoughts: we can explain the winter peak increase as a result of the floodplain storage, the lake storage starts building up with the rising limb and then when it maxes it serves to add to peak flow it seems.  

2023-12-1 upon further review, when I do the WB chck it seems like the restoration should be lower

In [ ]:
lak_avg = lak_out0.loc[plt_dates].mean(numeric_only=True)
lak_avg['SW-Inflw'] - lak_avg['SW-Outflw']
# under the baseline scenario 6k m3/day on avg are lost from lake inflow to outflow
# under restoration 18k m3/day on avg are lost from lake inflow to lake outflow
# this proves that there shouldn't be a big jump up in the restoration outflow

- Lake out matches the outflow from segment 51 which means it's operating properly.
- outflow after the lake is smaller than the uppermost inflow at peak flow but greater than it after peak flow during the drain out of the floodplain.
- 

In [ ]:
fig, ax = plt.subplots()
# lak_out0.loc[plt_dates].plot(y='SW-Outflw', ax=ax, label='lake out')
sfrdf0[sfrdf0.segment==51].loc[plt_dates].plot(y='Qout', ax=ax, label='51 out')
sfrdf0[sfrdf0.segment==50].loc[plt_dates].plot(y='Qout', ax=ax, label='50 out')
sfrdf0[sfrdf0.segment==30].loc[plt_dates].plot(y='Qout', ax=ax, label='30 out')
sfrdf0[sfrdf0.segment==52].loc[plt_dates].plot(y='Qout', ax=ax, label='52 out')

In [ ]:
fig,ax = plt.subplots()
sfr_avg0 = sfrdf0.loc[plt_dates].groupby('Total distance (m)').mean(numeric_only=True).reset_index()
sfr_avg0.plot(x='Total distance (m)',y='Qout',ax =ax, label=label_restoration)

sfr_avg = sfrdf.loc[plt_dates].groupby('Total distance (m)').mean(numeric_only=True).reset_index()
sfr_avg.plot(x='Total distance (m)',y='Qout',ax =ax, label=label_baseline)

# plt.ylim(5E5,7E5)


The longitudinal streamflow plot shows there is a discontuity in the restoration scenario below the floodplain as the streamflow is higher than the upstream inflow which indicates that the floodplain is gaining water.
- segment 51 diverts flow from the lake and feeds into 52.

**Error is at lake inflow (segment 31/32)**
Segment 31 has the full diversion above the flow threshold and segment 32 should be pulling half of that to go back to the main channel. BUT the full diversion is being passed to the lake as inflow so half of the diversio is being added back as an extra flow.